<a href="https://colab.research.google.com/github/ipeirotis/datasets/blob/master/notebooks/NYPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NYPD Dataset

Dataset description at 
https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i



| Column | Description |
|--------|-------------------|
| CMPLNT_NUM |  Randomly generated persistent ID for each complaint  |  
| ADDR_PCT_CD |  The precinct in which the incident occurred |  
| BORO |  The name of the borough in which the incident occurred |  
| CMPLNT_FR_DT |  Exact date of occurrence for the reported event (or starting date of occurrence, if CMPLNT_TO_DT exists) |  
| CMPLNT_FR_TM |  Exact time of occurrence for the reported event (or starting time of occurrence, if CMPLNT_TO_TM exists) |  
| CMPLNT_TO_DT |  Ending date of occurrence for the reported event, if exact time of occurrence is unknown |  
| CMPLNT_TO_TM |  Ending time of occurrence for the reported event, if exact time of occurrence is unknown |  
| CRM_ATPT_CPTD_CD |  Indicator of whether crime was successfully completed or attempted, but failed or was interrupted prematurely |  
| HADEVELOPT |  Name of NYCHA housing development of occurrence, if applicable |  
| HOUSING_PSA |  Development Level Code |  
| JURISDICTION_CODE |  Jurisdiction responsible for incident. Either internal, like Police(0), Transit(1), and Housing(2); or external(3), like Correction, Port Authority, etc. |  
| JURIS_DESC |  Description of the jurisdiction code |  
| KY_CD |  Three digit offense classification code |  
| LAW_CAT_CD |  Level of offense: felony, misdemeanor, violation  |  
| LOC_OF_OCCUR_DESC |  Specific location of occurrence in or around the premises; inside, opposite of, front of, rear of |  
| OFNS_DESC |  Description of offense corresponding with key code |  
| PARKS_NM |  Name of NYC park, playground or greenspace of occurrence, if applicable (state parks are not included) |  
| PATROL_BORO |  The name of the patrol borough in which the incident occurred |  
| PD_CD |  Three digit internal classification code (more granular than Key Code) |  
| PD_DESC |  Description of internal classification corresponding with PD code (more granular than Offense Description) |  
| PREM_TYP_DESC |  Specific description of premises; grocery store, residence, street, etc. |  
| RPT_DT |  Date event was reported to police  |  
| STATION_NAME |  Transit station name |  
| SUSP_AGE_GROUP |  Suspect’s Age Group |  
| SUSP_RACE |  Suspect’s Race Description |  
| SUSP_SEX |  Suspect’s Sex Description |  
| TRANSIT_DISTRICT |  Transit district in which the offense occurred. |  
| VIC_AGE_GROUP |  Victim’s Age Group |  
| VIC_RACE |  Victim’s Race Description |  
| VIC_SEX |  Victim’s Sex Description |  
| X_COORD_CD |  X-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104) |  
| Y_COORD_CD |  Y-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104) |  
| Latitude |  Midblock Latitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326)  |  
| Longitude |  Midblock Longitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326) |


In [2]:
import pandas as pd
import numpy as np

In [3]:
# We load everything as an object/string, because some data types (e.g., some IDs)
# are recognized as decimals, and it is a mess to restore them back
# So we will do all the conversions ourselves later on

# From https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i/data
!curl 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD' -o nypd.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2391M    0 2391M    0     0  3233k      0 --:--:--  0:12:37 --:--:-- 4223k


In [163]:
%%time
df = pd.read_csv('nypd.csv', low_memory = True, dtype='object')

CPU times: user 2min 16s, sys: 17.8 s, total: 2min 34s
Wall time: 2min 12s


In [164]:
# We load directly from the URL
# url = 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD'
# df = pd.read_csv(url, low_memory = True, dtype='object')


In [165]:
len(df)

7825499

In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7825499 entries, 0 to 7825498
Data columns (total 35 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   CMPLNT_NUM         object
 1   CMPLNT_FR_DT       object
 2   CMPLNT_FR_TM       object
 3   CMPLNT_TO_DT       object
 4   CMPLNT_TO_TM       object
 5   ADDR_PCT_CD        object
 6   RPT_DT             object
 7   KY_CD              object
 8   OFNS_DESC          object
 9   PD_CD              object
 10  PD_DESC            object
 11  CRM_ATPT_CPTD_CD   object
 12  LAW_CAT_CD         object
 13  BORO_NM            object
 14  LOC_OF_OCCUR_DESC  object
 15  PREM_TYP_DESC      object
 16  JURIS_DESC         object
 17  JURISDICTION_CODE  object
 18  PARKS_NM           object
 19  HADEVELOPT         object
 20  HOUSING_PSA        object
 21  X_COORD_CD         object
 22  Y_COORD_CD         object
 23  SUSP_AGE_GROUP     object
 24  SUSP_RACE          object
 25  SUSP_SEX           object
 26  TRANSIT_DISTRI

## Data Cleaning

In [167]:
# These columns are redundant
to_drop = ['Lat_Lon','X_COORD_CD','Y_COORD_CD']
# We created the CMPLNT_FR and CMPLNT_TO columns
# We have the longitude and latitude so the other coordinates are not needed
df = df.drop(to_drop, axis='columns')

###  CMPLNT_NUM         object   

In [168]:
     
# Drop cases with duplicated complaint numbers
key_cnt = df.CMPLNT_NUM.value_counts()
key_cnt [ key_cnt>1 ]
df = df[ ~df.CMPLNT_NUM.isin( key_cnt [ key_cnt>1 ].index.values ) ]

In [169]:
df.CMPLNT_NUM = df.CMPLNT_NUM.astype('int32')

### CMPLNT_FR_DT       object
### CMPLNT_FR_TM       object
### CMPLNT_TO_DT       object
### CMPLNT_TO_TM       object

In [170]:
# There are a few rows that contain year 1015, 1016, ... that trigger an error during date conversion
# We replace all years written as 10XX with 20XX
# Note the usage of regular expressions
df.CMPLNT_FR_DT.replace(to_replace = '(\d\d)/(\d\d)/10(\d\d)', value=r'\1/\2/20\3', inplace = True, regex=True )
df.CMPLNT_TO_DT.replace(to_replace = '(\d\d)/(\d\d)/10(\d\d)', value=r'\1/\2/20\3', inplace = True, regex=True )

# Similarly, a few hours are written as 24:00:00, which also triggers errors.
# We fix these hours
df.CMPLNT_FR_TM.replace(to_replace = '24:00:00', value='00:00:00', inplace = True)
df.CMPLNT_TO_TM.replace(to_replace = '24:00:00', value='00:00:00', inplace = True)

# Convert the two separate date and time columns into single datetime columns
df['CMPLNT_FR'] = pd.to_datetime(df.CMPLNT_FR_DT + ' ' + df.CMPLNT_FR_TM, format='%m/%d/%Y %H:%M:%S', cache=True)
df['CMPLNT_TO'] = pd.to_datetime(df.CMPLNT_TO_DT + ' ' + df.CMPLNT_TO_TM, format='%m/%d/%Y %H:%M:%S', cache=True)

# These columns are redundant
to_drop = ['CMPLNT_FR_DT','CMPLNT_TO_DT','CMPLNT_FR_TM','CMPLNT_TO_TM']
# We created the CMPLNT_FR and CMPLNT_TO columns
# We have the longitude and latitude so the other coordinates are not needed
df = df.drop(to_drop, axis='columns')

In [171]:
df.CMPLNT_FR.isnull().sum()

702

In [172]:
df.CMPLNT_TO.isnull().sum()

1744179

In [173]:
df = df [ ~df.CMPLNT_FR.isnull() ]

###  ADDR_PCT_CD        object

In [174]:

df.ADDR_PCT_CD.replace(to_replace = '-99', value='99', inplace = True)
# df = df [ ~df.ADDR_PCT_CD.isnull() ]
df.ADDR_PCT_CD = pd.Categorical(df.ADDR_PCT_CD)

###  RPT_DT             object

In [175]:


# Convert RPT_DT to date
df.RPT_DT = pd.to_datetime(df.RPT_DT, format="%m/%d/%Y", cache=True)

### 7   KY_CD              object
### 8   OFNS_DESC          object

In [176]:
df.OFNS_DESC.replace(to_replace = 'KIDNAPPING', value='KIDNAPPING & RELATED OFFENSES', inplace = True)
df.OFNS_DESC.replace(to_replace = 'KIDNAPPING AND RELATED OFFENSES', value='KIDNAPPING & RELATED OFFENSES', inplace = True)

df.OFNS_DESC.replace(to_replace = 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED', value='OTHER STATE LAWS (NON PENAL LAW)', inplace = True)
df.OFNS_DESC.replace(to_replace = 'OTHER STATE LAWS (NON PENAL LA', value='OTHER STATE LAWS (NON PENAL LAW)', inplace = True)

df.OFNS_DESC.replace(to_replace = 'ENDAN WELFARE INCOMP', value='OFFENSES RELATED TO CHILDREN', inplace = True)

df.OFNS_DESC.replace(to_replace = 'THEFT OF SERVICES', value='OTHER OFFENSES RELATED TO THEF', inplace = True)

df.OFNS_DESC.replace(to_replace = 'NYS LAWS-UNCLASSIFIED VIOLATION', value='OTHER STATE LAWS', inplace = True)

df.OFNS_DESC.replace(to_replace = 'FELONY SEX CRIMES', value='SEX CRIMES', inplace = True)

df.loc[df.KY_CD=='120','OFNS_DESC'] ='CHILD ABANDONMENT/NON SUPPORT'

df.loc[df.KY_CD=='125','OFNS_DESC'] ='NYS LAWS-UNCLASSIFIED FELONY'

offenses = df[ ["KY_CD", "OFNS_DESC"] ].drop_duplicates().dropna()
offenses['KY_CD'] = pd.Categorical(pd.to_numeric(offenses['KY_CD'] ).astype(int))
offenses = offenses.set_index("KY_CD")
offenses = offenses.sort_index()
offenses = offenses.reset_index()
offenses


,KY_CD,OFNS_DESC
0,101,MURDER & NON-NEGL. MANSLAUGHTER
1,102,HOMICIDE-NEGLIGENT-VEHICLE
2,103,"HOMICIDE-NEGLIGENT,UNCLASSIFIE"
3,104,RAPE
4,105,ROBBERY
...,...,...
69,676,NEW YORK CITY HEALTH CODE
70,677,OTHER STATE LAWS
71,678,MISCELLANEOUS PENAL LAW
72,685,ADMINISTRATIVE CODES


In [177]:
df.KY_CD = pd.Categorical(df.KY_CD)

In [178]:
df = df.drop('OFNS_DESC', axis='columns')

### 9   PD_CD              object
### 10  PD_DESC            object

In [179]:



df.loc[df.PD_CD=='694','PD_DESC'] ='INCEST'

df.loc[df.PD_CD=='234','PD_DESC'] ='BURGLARY,UNKNOWN TIME'

internal = df[ ["PD_CD", "PD_DESC"] ].drop_duplicates().dropna()
internal['PD_CD'] = pd.Categorical(pd.to_numeric(internal['PD_CD'] ).astype(int))
internal = internal.set_index("PD_CD")
internal = internal.sort_index()
internal = internal.reset_index()
internal

,PD_CD,PD_DESC
0,100,STALKING COMMIT SEX OFFENSE
1,101,ASSAULT 3
2,102,ASSAULT SCHOOL SAFETY AGENT
3,103,ASSAULT TRAFFIC AGENT
4,104,VEHICULAR ASSAULT (INTOX DRIVE
...,...,...
427,916,LEAVING SCENE-ACCIDENT-PERSONA
428,918,RECKLESS DRIVING
429,922,"TRAFFIC,UNCLASSIFIED MISDEMEAN"
430,969,"TRAFFIC,UNCLASSIFIED INFRACTIO"


In [180]:
df.PD_CD.isnull().sum()

6751

In [181]:
# df = df[~df.PD_CD.isnull()]

In [182]:
df.PD_CD = pd.Categorical(df.PD_CD)

In [183]:
df = df.drop('PD_DESC', axis='columns')

### 11  CRM_ATPT_CPTD_CD   object

In [184]:
df.CRM_ATPT_CPTD_CD.value_counts()

COMPLETED    7684519
ATTEMPTED     132187
Name: CRM_ATPT_CPTD_CD, dtype: int64

In [185]:
df.CRM_ATPT_CPTD_CD = pd.Categorical(df.CRM_ATPT_CPTD_CD)

In [186]:
df.CRM_ATPT_CPTD_CD.isnull().sum()

167

In [187]:
# df = df [ ~df.CRM_ATPT_CPTD_CD.isnull() ]

### 12  LAW_CAT_CD         object

In [188]:
df.LAW_CAT_CD.value_counts()

MISDEMEANOR    4361634
FELONY         2421052
VIOLATION      1034187
Name: LAW_CAT_CD, dtype: int64

In [189]:
df.LAW_CAT_CD = pd.Categorical(df.LAW_CAT_CD)

### 16  JURIS_DESC         object
### 17  JURISDICTION_CODE  object

In [190]:
df.JURISDICTION_CODE.isnull().sum()

6751

In [191]:
# df = df[ ~df.JURISDICTION_CODE.isnull() ]

jusridiction = df[ ["JURISDICTION_CODE", "JURIS_DESC", ] ].drop_duplicates().dropna()
jusridiction['JURISDICTION_CODE'] = pd.to_numeric(jusridiction['JURISDICTION_CODE'] )
jusridiction['JURISDICTION_CODE'] = jusridiction['JURISDICTION_CODE'].astype(int)
jusridiction = jusridiction.set_index("JURISDICTION_CODE")
jusridiction = jusridiction.sort_index()
jusridiction = jusridiction.reset_index()
jusridiction

,JURISDICTION_CODE,JURIS_DESC
0,0,N.Y. POLICE DEPT
1,1,N.Y. TRANSIT POLICE
2,2,N.Y. HOUSING POLICE
3,3,PORT AUTHORITY
4,4,TRI-BORO BRDG TUNNL
5,6,LONG ISLAND RAILRD
6,7,AMTRACK
7,8,CONRAIL
8,9,STATN IS RAPID TRANS
9,11,N.Y. STATE POLICE


In [192]:
df.JURISDICTION_CODE = pd.Categorical(df.JURISDICTION_CODE)


In [193]:
df = df.drop('JURIS_DESC', axis='columns')

###  13  BORO_NM            object

In [194]:
df.BORO_NM.value_counts()

BROOKLYN         2310601
MANHATTAN        1880269
BRONX            1692104
QUEENS           1559868
STATEN ISLAND     361563
Name: BORO_NM, dtype: int64

In [195]:
df.BORO_NM.isnull().sum()

12468

In [196]:
# df = df[~df.BORO_NM.isnull()]

In [197]:
df.BORO_NM = pd.Categorical(df.BORO_NM)

### 23  SUSP_AGE_GROUP     object
### 32  VIC_AGE_GROUP      object

In [198]:
# Both columns have a lot of noisy entries. We keep only the dominant groups, and also define an order
df.SUSP_AGE_GROUP = pd.Categorical(df.SUSP_AGE_GROUP, ordered=True, categories=['<18', '18-24',  '25-44', '45-64', '65+'])
df.VIC_AGE_GROUP = pd.Categorical(df.VIC_AGE_GROUP, ordered=True, categories=['<18', '18-24',  '25-44', '45-64', '65+'])


### 24  SUSP_RACE          object
### 25  SUSP_SEX           object

### 33  VIC_RACE           object
### 34  VIC_SEX            object

In [199]:
df.VIC_SEX.isnull().sum()

308

In [200]:
df.VIC_SEX.value_counts()

F    3064853
M    2590144
E    1187478
D     974086
U          4
Name: VIC_SEX, dtype: int64

In [201]:
# df = df[~df.VIC_SEX.isnull()]

df.VIC_SEX.replace(to_replace = 'U', value=np.nan, inplace = True)

In [202]:
df.VIC_RACE.isnull().sum()

388

In [203]:
df.VIC_RACE.value_counts()

UNKNOWN                           2556088
BLACK                             1886404
WHITE                             1333985
WHITE HISPANIC                    1271732
ASIAN / PACIFIC ISLANDER           463619
BLACK HISPANIC                     269739
AMERICAN INDIAN/ALASKAN NATIVE      34887
OTHER                                  31
Name: VIC_RACE, dtype: int64

In [204]:
df.VIC_RACE.replace(to_replace = 'OTHER', value='UNKNOWN', inplace = True)

In [205]:
# df = df[~df.VIC_RACE.isnull()]

In [206]:
df.SUSP_SEX.value_counts()

M    2600924
F     812467
U     758148
Name: SUSP_SEX, dtype: int64

In [207]:
# U is unknown, same is NULL.
df.SUSP_SEX.replace(to_replace = 'U', value=np.nan, inplace = True)

In [208]:
df.SUSP_RACE.value_counts()

BLACK                             1600635
UNKNOWN                           1143340
WHITE HISPANIC                     724965
WHITE                              462776
BLACK HISPANIC                     221115
ASIAN / PACIFIC ISLANDER           139321
AMERICAN INDIAN/ALASKAN NATIVE      12637
OTHER                                  11
Name: SUSP_RACE, dtype: int64

In [209]:
# Very small amount of OTHER values
df.SUSP_RACE.replace(to_replace = 'OTHER', value='UNKNOWN', inplace = True)



In [210]:
df.SUSP_RACE = pd.Categorical(df.SUSP_RACE)
df.SUSP_SEX = pd.Categorical(df.SUSP_SEX)
df.VIC_RACE = pd.Categorical(df.VIC_RACE)
df.VIC_SEX = pd.Categorical(df.VIC_SEX)

In [211]:
df.dtypes

CMPLNT_NUM                    int32
ADDR_PCT_CD                category
RPT_DT               datetime64[ns]
KY_CD                      category
PD_CD                      category
CRM_ATPT_CPTD_CD           category
LAW_CAT_CD                 category
BORO_NM                    category
LOC_OF_OCCUR_DESC            object
PREM_TYP_DESC                object
JURISDICTION_CODE          category
PARKS_NM                     object
HADEVELOPT                   object
HOUSING_PSA                  object
SUSP_AGE_GROUP             category
SUSP_RACE                  category
SUSP_SEX                   category
TRANSIT_DISTRICT             object
Latitude                     object
Longitude                    object
PATROL_BORO                  object
STATION_NAME                 object
VIC_AGE_GROUP              category
VIC_RACE                   category
VIC_SEX                    category
CMPLNT_FR            datetime64[ns]
CMPLNT_TO            datetime64[ns]
dtype: object

###  14  LOC_OF_OCCUR_DESC  object

In [212]:
df.LOC_OF_OCCUR_DESC.value_counts()

INSIDE         3984710
FRONT OF       1841828
OPPOSITE OF     204732
REAR OF         164986
OUTSIDE           4156
Name: LOC_OF_OCCUR_DESC, dtype: int64

In [213]:
df.LOC_OF_OCCUR_DESC.isnull().sum()

1616461

In [214]:
df.LOC_OF_OCCUR_DESC = pd.Categorical(df.LOC_OF_OCCUR_DESC)

### Latitude                     object
### Longitude                    object

In [215]:
# !sudo apt-get update
!sudo apt-get install python3-rtree
!sudo pip3 install geopandas descartes shapely ngram # matplotlib==3.1.3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-rtree is already the newest version (0.8.3+ds-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [216]:
import geopandas as gpd

In [217]:
df.Latitude = pd.to_numeric(df.Latitude, downcast='float')
df.Longitude  = pd.to_numeric(df.Longitude, downcast='float')

In [218]:
%%time
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

CPU times: user 13min 43s, sys: 23.8 s, total: 14min 6s
Wall time: 11min 48s


In [219]:
shapefile_url = 'https://data.cityofnewyork.us/api/geospatial/cpf4-rkhq?method=export&format=Shapefile'
df_nyc = gpd.GeoDataFrame.from_file(shapefile_url)

In [220]:
%%time
# Match each accident with a neighborhood. 
# Will take ~30 mins to run
# This is done with left join, 
# so we preserve all the data points
# but we know which ones are not matching with the shapefile
gdf.crs = df_nyc.crs
gdf = gpd.sjoin(gdf, df_nyc, how='left')


CPU times: user 45min 16s, sys: 48.6 s, total: 46min 4s
Wall time: 38min 30s


In [221]:
# We keep only boro_name and ntaname
todrop = [
    'index_right', 'boro_code', 'county_fip', 
    'shape_area', 'shape_leng'
]

gdf = gdf.drop(todrop, axis='columns')

# Rename the columns
gdf = gdf.rename({
    'boro_name': 'BOROUGH',
    'ntaname': 'NEIGHBORHOOD',
    'ntacode': 'NEIGHBORHOOD_CODE',
}, axis='columns')

In [222]:
gdf['BOROUGH'] = gdf['BOROUGH'].str.upper()

In [223]:
# Mark as NULL all the lon/lat entries outside the NYC area
gdf.loc[gdf.BOROUGH.isnull(), 'Latitude'] = None
gdf.loc[gdf.BOROUGH.isnull(), 'Longitude'] = None

In [224]:
# Mark as NULL all the lon/lat entries that generate inconsistencies
mask = gdf.query('BOROUGH != BORO_NM and Latitude==Latitude and Longitude==Longitude').CMPLNT_NUM.values
condition = gdf.CMPLNT_NUM.isin(mask)

gdf.loc[condition, 'Latitude'] = None
gdf.loc[condition, 'Longitude'] = None

In [225]:
# Delete the cases where the reported and detected boroughs are different
# to_delete = sorted(set(to_delete))
# gdf = gdf [ ~gdf.CMPLNT_NUM.isin(to_delete) ]

In [226]:
gdf = gdf.drop('geometry', axis='columns')

In [227]:
df = pd.DataFrame(gdf)

In [228]:
df.BORO_NM.value_counts()

BROOKLYN         2310601
MANHATTAN        1880269
BRONX            1692104
QUEENS           1559868
STATEN ISLAND     361563
Name: BORO_NM, dtype: int64

In [229]:
#Temporarily, we drop these. We will add them back in the future
df.drop(['BOROUGH','NEIGHBORHOOD','NEIGHBORHOOD_CODE'], axis='columns', inplace=True)

### TRANSIT_DISTRICT

In [230]:
df.TRANSIT_DISTRICT.value_counts()


4     28870
2     21342
1     16853
3     16702
20    15351
33    15095
12    12937
11    12068
32    11637
30    11070
34     7543
23     2829
Name: TRANSIT_DISTRICT, dtype: int64

In [231]:
len(df) - df.TRANSIT_DISTRICT.isnull().sum()

172297

In [232]:
df.drop('TRANSIT_DISTRICT', axis='columns', inplace=True)


### PREM_TYP_DESC

In [233]:
df.PREM_TYP_DESC.value_counts()

STREET                        2477345
RESIDENCE - APT. HOUSE        1671518
RESIDENCE-HOUSE                771962
RESIDENCE - PUBLIC HOUSING     585016
OTHER                          208864
                               ...   
MAILBOX INSIDE                    889
CEMETERY                          885
LOAN COMPANY                      520
TRAMWAY                           145
DAYCARE FACILITY                   71
Name: PREM_TYP_DESC, Length: 74, dtype: int64

In [234]:
df.PREM_TYP_DESC.isnull().sum()

41979

In [235]:
# df = df [~df.PREM_TYP_DESC.isnull()]

In [236]:
df.PREM_TYP_DESC = pd.Categorical(df.PREM_TYP_DESC)

In [237]:
df.PARKS_NM.value_counts()

CENTRAL PARK                            1856
FLUSHING MEADOWS CORONA PARK            1530
CONEY ISLAND BEACH & BOARDWALK          1160
WASHINGTON SQUARE PARK                  1061
RIVERSIDE PARK                           680
                                        ... 
GIVANS CREEK WOODS                         1
UDALL'S PARK PRESERVE                      1
O'SULLIVAN PLAZA                           1
HANCOCK T AND T                            1
UNNAMED PARK ON CLEARVIEW EXPRESSWAY       1
Name: PARKS_NM, Length: 1233, dtype: int64

In [238]:
df.PARKS_NM.value_counts().sum()

30798

In [239]:
df.drop('PARKS_NM', axis='columns', inplace=True)



 19  HADEVELOPT         object


In [240]:
df.HADEVELOPT.value_counts()

CASTLE HILL                 7546
VAN DYKE I                  6104
MARCY                       5595
GRANT                       5293
BUTLER                      5226
                            ... 
JEFFERSON                      1
344 EAST 28TH STREET           1
BRONXCHESTER                   1
FRANKLIN AVENUE III MHOP       1
1010 EAST 178TH STREET         1
Name: HADEVELOPT, Length: 279, dtype: int64

In [241]:
df.drop('HADEVELOPT', axis='columns', inplace=True)


 20  HOUSING_PSA        object



In [242]:
df.HOUSING_PSA.value_counts()

670      7781
887      7690
845      7060
720      6850
632      6644
         ... 
33107       1
17910       1
2507        1
2346        1
8074        1
Name: HOUSING_PSA, Length: 5110, dtype: int64

In [243]:
df.HOUSING_PSA.value_counts().sum()

598710

In [244]:
df.drop('HOUSING_PSA', axis='columns', inplace=True)

 30  PATROL_BORO        object


In [245]:
df.PATROL_BORO.value_counts()

PATROL BORO BRONX            1692880
PATROL BORO BKLYN SOUTH      1158352
PATROL BORO BKLYN NORTH      1153752
PATROL BORO MAN SOUTH         946144
PATROL BORO MAN NORTH         933895
PATROL BORO QUEENS NORTH      809285
PATROL BORO QUEENS SOUTH      753675
PATROL BORO STATEN ISLAND     361682
Name: PATROL_BORO, dtype: int64

In [246]:
df.PATROL_BORO = pd.Categorical(df.PATROL_BORO)

In [247]:
df.PATROL_BORO.isnull().sum()

7208

In [248]:
# df = df[~df.PATROL_BORO.isnull()] 

 31  STATION_NAME       object

In [249]:
df.STATION_NAME.value_counts()

125 STREET                        8540
14 STREET                         4681
42 ST.-PORT AUTHORITY BUS TERM    4068
34 ST.-PENN STATION               4033
116 STREET                        3438
                                  ... 
DISTRICT 30 OFFICE                  19
DISTRICT 12 OFFICE                  19
DISTRICT 34 OFFICE                  14
DISTRICT 23 OFFICE                   6
OFF-SYSTEM                           4
Name: STATION_NAME, Length: 372, dtype: int64

In [250]:
df.STATION_NAME.isnull().sum()

7644576

In [251]:
df.drop('STATION_NAME', axis='columns', inplace=True)

In [252]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7816873 entries, 0 to 7825498
Data columns (total 22 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CMPLNT_NUM         int32         
 1   ADDR_PCT_CD        category      
 2   RPT_DT             datetime64[ns]
 3   KY_CD              category      
 4   PD_CD              category      
 5   CRM_ATPT_CPTD_CD   category      
 6   LAW_CAT_CD         category      
 7   BORO_NM            category      
 8   LOC_OF_OCCUR_DESC  category      
 9   PREM_TYP_DESC      category      
 10  JURISDICTION_CODE  category      
 11  SUSP_AGE_GROUP     category      
 12  SUSP_RACE          category      
 13  SUSP_SEX           category      
 14  Latitude           float32       
 15  Longitude          float32       
 16  PATROL_BORO        category      
 17  VIC_AGE_GROUP      category      
 18  VIC_RACE           category      
 19  VIC_SEX            category      
 20  CMPLNT_FR          datet

## Data exploration

In this part we check the different values that appear in the columns. When we detect noisy results, we delete the corresponding values. In fact, many of the operations that are performed above, in the 'data cleaning' section, are the result of observations that we make here. 

In [253]:
# Find the unique values in each column
# 
# df.describe(include = [np.object, 'category']).T['unique']
unique = df.describe(include = 'all').T['unique'].sort_values()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  after removing the cwd from sys.path.


In [254]:
unique

CRM_ATPT_CPTD_CD           2
SUSP_SEX                   2
LAW_CAT_CD                 3
VIC_SEX                    4
BORO_NM                    5
LOC_OF_OCCUR_DESC          5
VIC_AGE_GROUP              5
SUSP_AGE_GROUP             5
VIC_RACE                   7
SUSP_RACE                  7
PATROL_BORO                8
JURISDICTION_CODE         25
KY_CD                     74
PREM_TYP_DESC             74
ADDR_PCT_CD               77
PD_CD                    432
RPT_DT                  5844
CMPLNT_FR            2210055
CMPLNT_TO            2443005
CMPLNT_NUM               NaN
Latitude                 NaN
Longitude                NaN
Name: unique, dtype: object

In [255]:
for column in unique.index:
    if unique[column] < 200:
        print(df[column].value_counts())
        print("=====")

COMPLETED    7684519
ATTEMPTED     132187
Name: CRM_ATPT_CPTD_CD, dtype: int64
=====
M    2600924
F     812467
Name: SUSP_SEX, dtype: int64
=====
MISDEMEANOR    4361634
FELONY         2421052
VIOLATION      1034187
Name: LAW_CAT_CD, dtype: int64
=====
F    3064853
M    2590144
E    1187478
D     974086
Name: VIC_SEX, dtype: int64
=====
BROOKLYN         2310601
MANHATTAN        1880269
BRONX            1692104
QUEENS           1559868
STATEN ISLAND     361563
Name: BORO_NM, dtype: int64
=====
INSIDE         3984710
FRONT OF       1841828
OPPOSITE OF     204732
REAR OF         164986
OUTSIDE           4156
Name: LOC_OF_OCCUR_DESC, dtype: int64
=====
25-44    2597750
45-64    1357000
18-24     788503
<18       353308
65+       280834
Name: VIC_AGE_GROUP, dtype: int64
=====
25-44    1128505
18-24     417739
45-64     400159
<18       115218
65+        34299
Name: SUSP_AGE_GROUP, dtype: int64
=====
UNKNOWN                           2556119
BLACK                             1886404
WHITE    

In [256]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7816873 entries, 0 to 7825498
Data columns (total 22 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CMPLNT_NUM         int32         
 1   ADDR_PCT_CD        category      
 2   RPT_DT             datetime64[ns]
 3   KY_CD              category      
 4   PD_CD              category      
 5   CRM_ATPT_CPTD_CD   category      
 6   LAW_CAT_CD         category      
 7   BORO_NM            category      
 8   LOC_OF_OCCUR_DESC  category      
 9   PREM_TYP_DESC      category      
 10  JURISDICTION_CODE  category      
 11  SUSP_AGE_GROUP     category      
 12  SUSP_RACE          category      
 13  SUSP_SEX           category      
 14  Latitude           float32       
 15  Longitude          float32       
 16  PATROL_BORO        category      
 17  VIC_AGE_GROUP      category      
 18  VIC_RACE           category      
 19  VIC_SEX            category      
 20  CMPLNT_FR          datet

In [257]:
df['PREM_TYP_DESC'].value_counts().count()

74

In [258]:
# All columns, except for the dates and spatial coordinates, are categorical
# Columns with less than a few thousand unique values are good candidates 
# for ENUMs in the database given that the dataset is static.
# Also, in Pandas the internal representation becomes much more efficient
# as the Categoricals are stored as integers and not as strings
for column in unique.index:
    if column == 'RPT_DT':
        continue
    if df[column].value_counts().count() < 1000:
      df[column] = pd.Categorical(df[column])

In [259]:
# With all the proper data typing the dataset went down in size from 1.9Gb+ to 425Mb.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7816873 entries, 0 to 7825498
Data columns (total 22 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CMPLNT_NUM         int32         
 1   ADDR_PCT_CD        category      
 2   RPT_DT             datetime64[ns]
 3   KY_CD              category      
 4   PD_CD              category      
 5   CRM_ATPT_CPTD_CD   category      
 6   LAW_CAT_CD         category      
 7   BORO_NM            category      
 8   LOC_OF_OCCUR_DESC  category      
 9   PREM_TYP_DESC      category      
 10  JURISDICTION_CODE  category      
 11  SUSP_AGE_GROUP     category      
 12  SUSP_RACE          category      
 13  SUSP_SEX           category      
 14  Latitude           float32       
 15  Longitude          float32       
 16  PATROL_BORO        category      
 17  VIC_AGE_GROUP      category      
 18  VIC_RACE           category      
 19  VIC_SEX            category      
 20  CMPLNT_FR          datet

In [260]:
df.memory_usage()

Index                62534984
CMPLNT_NUM           31267492
ADDR_PCT_CD           7819593
RPT_DT               62534984
KY_CD                 7819569
PD_CD                15653754
CRM_ATPT_CPTD_CD      7816997
LAW_CAT_CD            7817005
BORO_NM               7817085
LOC_OF_OCCUR_DESC     7817085
PREM_TYP_DESC         7819569
JURISDICTION_CODE     7817629
SUSP_AGE_GROUP        7817085
SUSP_RACE             7817229
SUSP_SEX              7816997
Latitude             31267492
Longitude            31267492
PATROL_BORO           7817237
VIC_AGE_GROUP         7817085
VIC_RACE              7817229
VIC_SEX               7817077
CMPLNT_FR            62534984
CMPLNT_TO            62534984
dtype: int64

In [261]:
df.dtypes

CMPLNT_NUM                    int32
ADDR_PCT_CD                category
RPT_DT               datetime64[ns]
KY_CD                      category
PD_CD                      category
CRM_ATPT_CPTD_CD           category
LAW_CAT_CD                 category
BORO_NM                    category
LOC_OF_OCCUR_DESC          category
PREM_TYP_DESC              category
JURISDICTION_CODE          category
SUSP_AGE_GROUP             category
SUSP_RACE                  category
SUSP_SEX                   category
Latitude                    float32
Longitude                   float32
PATROL_BORO                category
VIC_AGE_GROUP              category
VIC_RACE                   category
VIC_SEX                    category
CMPLNT_FR            datetime64[ns]
CMPLNT_TO            datetime64[ns]
dtype: object

## Storing in a MySQL database

In [262]:
!sudo pip3 install -U -q PyMySQL sqlalchemy sql_magic

In [267]:
import os
from sqlalchemy import create_engine

conn_string = 'mysql+pymysql://{user}:{password}@{host}/?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org', 
    user = 'root',
    password = 'ae6jQniBS5muV27', #os.environ['MYSQL_PASSWORD'],
    encoding = 'utf8mb4')

engine = create_engine(conn_string)
con = engine.connect()

In [268]:
# con.close()

In [269]:
# Query to create a database
db_name = 'nypd'

sql = f"DROP DATABASE IF EXISTS {db_name}"
engine.execute(sql)

# Create a database
sql = f"CREATE DATABASE IF NOT EXISTS {db_name} DEFAULT CHARACTER SET 'utf8mb4'"
engine.execute(sql)


In [270]:

# And lets switch to the database
sql = f"USE {db_name}"
engine.execute(sql)


In [271]:
# In principle, we can let Pandas create the table, but we want to be a bit more predise
# with the data types, and we want to add documentation for each column
# from https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i


create_table_sql = '''
CREATE TABLE nypd (
  CMPLNT_NUM int,
  CMPLNT_FR datetime,
  CMPLNT_TO datetime,
  RPT_DT date,
  KY_CD SMALLINT,
  PD_CD SMALLINT,
  CRM_ATPT_CPTD_CD enum('COMPLETED','ATTEMPTED'),
  LAW_CAT_CD enum('FELONY','MISDEMEANOR','VIOLATION'),
  JURISDICTION_CODE SMALLINT,
  BORO_NM enum('BRONX','BROOKLYN','MANHATTAN','QUEENS','STATEN ISLAND'),
  ADDR_PCT_CD SMALLINT,
  LOC_OF_OCCUR_DESC enum('FRONT OF','INSIDE','OPPOSITE OF','OUTSIDE','REAR OF'),
  PATROL_BORO enum('PATROL BORO BRONX', 'PATROL BORO BKLYN SOUTH','PATROL BORO BKLYN NORTH','PATROL BORO MAN SOUTH','PATROL BORO MAN NORTH','PATROL BORO QUEENS NORTH','PATROL BORO QUEENS SOUTH','PATROL BORO STATEN ISLAND'),
  PREM_TYP_DESC varchar(30),
  SUSP_RACE enum('UNKNOWN', 'BLACK', 'WHITE', 'WHITE HISPANIC', 'ASIAN / PACIFIC ISLANDER', 'BLACK HISPANIC', 'AMERICAN INDIAN/ALASKAN NATIVE'),
  VIC_RACE enum('UNKNOWN', 'BLACK', 'WHITE', 'WHITE HISPANIC', 'ASIAN / PACIFIC ISLANDER', 'BLACK HISPANIC', 'AMERICAN INDIAN/ALASKAN NATIVE'),
  SUSP_AGE_GROUP enum('<18', '18-24',  '25-44', '45-64', '65+'),
  VIC_AGE_GROUP enum('<18', '18-24',  '25-44', '45-64', '65+'),
  SUSP_SEX enum('M', 'F'),
  VIC_SEX enum('M', 'F', 'E', 'D'),
  Latitude double,
  Longitude double,
  PRIMARY KEY (CMPLNT_NUM)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
engine.execute(create_table_sql)

In [272]:
# Create a table
# See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html for the documentation
from tqdm import tqdm
batchsize = 50000
batches = len(df) // batchsize + 1

t = tqdm(range(batches))

for i in t:
    # print("Batch:",i)
    # continue # Cannot execute this on Travis
    start = batchsize * i
    end = batchsize * (i+1)
    df[start:end].to_sql(
        name = 'nypd', 
        schema = db_name, 
        con = engine,
        if_exists = 'append',
        index = False, 
        chunksize = 1000)

100%|██████████| 157/157 [1:15:09<00:00, 28.73s/it]


In [273]:
engine.execute("CREATE INDEX ix_lat ON nypd.nypd(Latitude)")

In [274]:
engine.execute("CREATE INDEX ix_lon ON nypd.nypd(Longitude)")

In [275]:
engine.execute("CREATE INDEX ix_LAW_CAT_CD ON nypd.nypd(LAW_CAT_CD)")

In [276]:
engine.execute("CREATE INDEX ix_BORO_NM ON nypd.nypd(BORO_NM)")

In [277]:
engine.execute("CREATE INDEX ix_KY_CD ON nypd.nypd(KY_CD)")

In [278]:
engine.execute("CREATE INDEX ix_RPT_DT ON nypd.nypd(RPT_DT)")

In [279]:
engine.execute("CREATE INDEX ix_CMPLNT_FR ON nypd.nypd(CMPLNT_FR)")

In [280]:
engine.execute("DROP TABLE IF EXISTS offense_codes;")

create_table_sql = '''
CREATE TABLE offense_codes (
  KY_CD smallint,
  OFNS_DESC varchar(32),
  PRIMARY KEY (KY_CD)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
engine.execute(create_table_sql)

offenses.to_sql(
        name = 'offense_codes', 
        schema = db_name, 
        con = engine,
        if_exists = 'append',
        index = False)

In [281]:
engine.execute("DROP TABLE IF EXISTS jurisdiction_codes;")

create_table_sql = '''
CREATE TABLE jurisdiction_codes (
  JURISDICTION_CODE smallint,
  JURIS_DESC varchar(40),
  PRIMARY KEY (JURISDICTION_CODE)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
engine.execute(create_table_sql)


jusridiction.to_sql(
        name = 'jurisdiction_codes', 
        schema = db_name, 
        con = engine,
        if_exists = 'append',
        index = False)

In [282]:
internal.PD_DESC.str.len().max()


71

In [283]:
engine.execute("DROP TABLE IF EXISTS penal_codes;")

create_table_sql = '''
CREATE TABLE penal_codes (
  PD_CD smallint,
  PD_DESC varchar(80),
  PRIMARY KEY (PD_CD)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
engine.execute(create_table_sql)


internal.to_sql(
        name = 'penal_codes', 
        schema = db_name, 
        con = engine,
        if_exists = 'append',
        index = False)

## TODO

### Add the penal code data as a separate table

`!curl 'https://data.cityofnewyork.us/api/views/qgea-i56i/files/65f25845-1551-4d21-91dc-869c977cd93d?download=true&filename=PDCode_PenalLaw.xlsx' -o PDCode_PenalLaw.xlsx`

### Examine whether to normalize 

The fields 

  
PREM_TYP_DESC    
HADEVELOPT    
PARKS_NM                     

would be better off as foreign keys or enums. They take too much space as strings.